# **Guía de como Entrenar el Modelo, Guardarlo en un archivo pickle y como usarlo**

Para transformar el archivo `inside_airbnb_model.py` (que originalmente era un notebook) en un script Python que cargue el archivo `df_optimized.csv`, entrene el modelo v5 y lo guarde en formato pickle, sigue estos pasos:

### **Script Python (`train_model_v5.py`)**
```python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import RobustScaler
import pickle
from pathlib import Path

# --- Configuración de paths ---
processed_data_dir = Path("../data/processed/")
model_dir = Path("../models/")
model_dir.mkdir(parents=True, exist_ok=True)  # Crear directorio si no existe

# --- 1. Cargar datos procesados ---
df_optimized = pd.read_csv(processed_data_dir / "df_optimized.csv")

# --- 2. Transformación logarítmica del target (price) ---
y = np.log1p(df_optimized['price'])

# --- 3. Definir features (X) ---
# Eliminar columnas no relevantes para el modelo
X = df_optimized.drop(columns=['price', 'host_since_year'])  # Ajusta según tu CSV

# --- 4. Escalado numérico (opcional, pero recomendado) ---
numeric_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 'number_of_reviews']
scaler = RobustScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])

# --- 5. División train-test (para evaluación) ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 6. Entrenar modelo Random Forest v5 ---
model_v5 = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=2,
    max_features=0.5,
    max_samples=0.8,
    random_state=42,
    n_jobs=-1
)
model_v5.fit(X_train, y_train)

# --- 7. Evaluación del modelo ---
def print_metrics(y_true, y_pred, dataset_name):
    y_true_exp = np.expm1(y_true)
    y_pred_exp = np.expm1(y_pred)
    r2 = r2_score(y_true_exp, y_pred_exp)
    mae = mean_absolute_error(y_true_exp, y_pred_exp)
    print(f"\n📊 **Métricas para {dataset_name}**")
    print(f"R²: {r2:.4f} | MAE: {mae:.2f} EUR")

y_pred_train = model_v5.predict(X_train)
y_pred_test = model_v5.predict(X_test)
print_metrics(y_train, y_pred_train, "ENTRENAMIENTO")
print_metrics(y_test, y_pred_test, "PRUEBA")

# --- 8. Guardar modelo y scaler en pickle ---
with open(model_dir / "model_v5.pkl", 'wb') as f:
    pickle.dump(model_v5, f)

with open(model_dir / "scaler.pkl", 'wb') as f:
    pickle.dump(scaler, f)

print(f"\n✅ Modelo y scaler guardados en: {model_dir}/")
```

---

### **Explicación Paso a Paso**
1. **Carga de datos**:  
   - Lee el archivo `df_optimized.csv` desde la ruta `../data/processed/`.

2. **Preprocesamiento**:  
   - Aplica transformación logarítmica a `price` para manejar colas largas.  
   - Escala features numéricas con `RobustScaler` (protege contra outliers).

3. **Entrenamiento del modelo**:  
   - Usa los hiperparámetros óptimos del **modelo v5** (`n_estimators=300`, `max_features=0.5`, etc.).  
   - Evalúa con métricas en escala original (EUR).

4. **Exportación a pickle**:  
   - Guarda el modelo entrenado (`model_v5.pkl`) y el scaler (`scaler.pkl`) en la carpeta `../models/`.

---



### **Notas Clave**
- **Ajusta las columnas**: Verifica que `X` no incluya columnas irrelevantes o con leakage (como `log_price`).  
- **Entorno reproducible**: Usa `random_state=42` para resultados consistentes.  
- **Requisitos**: Asegúrate de tener las mismas versiones de librerías (`scikit-learn`, `pandas`, etc.) en entrenamiento y producción.  

Si necesitas adaptar algo específico (como las features usadas), modifica el script según tu caso.

-----

## **Como usar el Modelo**

Basado en el script de entrenamiento, aquí tienes cómo probar correctamente el modelo guardado (`random_forest_v5.pkl`):

### **Script completo para probar el modelo (`test_model.py`)**
```python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from sklearn.preprocessing import RobustScaler

# --- Configuración de paths ---
model_dir = Path("models/")
processed_data_dir = Path("data/processed/")

# --- 1. Cargar modelo y scaler ---
with open(model_dir / "random_forest_v5.pkl", 'rb') as f:
    model = pickle.load(f)
    
with open(model_dir / "scaler.pkl", 'rb') as f:
    scaler = pickle.load(f)

# --- 2. Cargar datos de prueba (o nuevos datos) ---
# Opción A: Usar datos de prueba guardados (si existen)
# df_test = pd.read_csv(processed_data_dir / "test_data.csv")

# Opción B: Crear datos de ejemplo (simulando entrada)
test_data = {
    'accommodates': [2, 4],  # Ejemplo: 2 personas
    'bathrooms': [1.0, 2.0],
    'bedrooms': [1, 2],
    'beds': [1, 3],
    'minimum_nights': [3, 2],
    'number_of_reviews': [10, 25],
    'review_scores_rating': [95, 80],
    'host_is_superhost': [1, 0],  # 1=True, 0=False
    'neighbourhood_density': [0.5, 0.3],
    'has_wifi': [1, 1],
    'has_air_conditioning': [1, 0],
    # Añade todas las features usadas en entrenamiento
}

X_new = pd.DataFrame(test_data)

# --- 3. Preprocesamiento igual que en entrenamiento ---
# Escalar features numéricas
numeric_features = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 
                   'minimum_nights', 'number_of_reviews']
X_new[numeric_features] = scaler.transform(X_new[numeric_features])

# --- 4. Predecir ---
pred_log = model.predict(X_new)  # Predicciones en escala logarítmica
pred_price = np.expm1(pred_log)  # Convertir a precio real (EUR)

# --- 5. Resultados ---
print("\n🔮 Predicciones:")
for i, price in enumerate(pred_price):
    print(f"Alojamiento {i+1}: ${price:.2f} EUR")

# --- 6. Opcional: Importancia de features ---
if hasattr(model, 'feature_importances_'):
    print("\n📊 Importancia de features:")
    features = X_new.columns
    importances = model.feature_importances_
    for feat, imp in sorted(zip(features, importances), key=lambda x: x[1], reverse=True):
        print(f"{feat}: {imp:.4f}")
```

---

### **¿Qué hace este script?**
1. **Carga el modelo y scaler** desde los archivos `.pkl`.
2. **Prepara datos de prueba**:
   - Puedes cargar un CSV con datos reales o simular datos como en el ejemplo.
3. **Aplica el mismo preprocesamiento**:
   - Escalado numérico con `RobustScaler` (¡igual que en entrenamiento!).
4. **Genera predicciones**:
   - Convierte los resultados de log(price) a EUR con `np.expm1()`.
5. **Muestra resultados**:
   - Precios predichos + importancia de cada feature (útil para debug).

---

### **Ejemplo de salida:**
```
🔮 Predicciones:
Alojamiento 1: $85.50 EUR
Alojamiento 2: $120.30 EUR

📊 Importancia de features:
bedrooms: 0.2543
accommodates: 0.1987
bathrooms: 0.1852
...
```

---

### **Requisitos para ejecutarlo**
1. Estructura de directorios:
   ```
   your_project/
   ├── models/
   │   ├── random_forest_v5.pkl
   │   └── scaler.pkl
   └── test_model.py
   ```

2. Bibliotecas:
   ```bash
   pip install pandas numpy scikit-learn
   ```

---

### **Consejos clave**
1. **Verifica las features**:  
   El DataFrame `X_new` debe tener exactamente las mismas columnas que usaste para entrenar (en mismo orden).

2. **Para producción**:  
   Guarda también la lista de columnas usadas en entrenamiento:
   ```python
   # Durante el entrenamiento:
   with open(model_dir / "feature_columns.pkl", 'wb') as f:
       pickle.dump(list(X_train.columns), f)
   
   # Durante la predicción:
   with open(model_dir / "feature_columns.pkl", 'rb') as f:
       expected_columns = pickle.load(f)
   X_new = X_new[expected_columns]  # Reordena columnas
   ```

3. **Si falla**:  
   Revisa que las versiones de `scikit-learn` sean consistentes entre entrenamiento y predicción.

## **1. Carga del Modelo.pkl y Generaración de Datos de Prueba Aleatorios y Consistentes**



Aquí tienes una solución completa que:
1. **Carga el modelo, scaler y columnas** desde los archivos `.pkl`.
2. **Genera datos de prueba aleatorios** basados en estadísticas reales del dataset original.
3. **Valida las columnas** para asegurar compatibilidad.

---

#### **Script (`test_model_with_random_data.py`)**
```python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import random

# --- Configuración de paths ---
PROJECT_ROOT = Path(__file__).parent.parent  # Ajusta según tu estructura
MODEL_PATH = PROJECT_ROOT / "models" / "random_forest_v5.pkl"
SCALER_PATH = PROJECT_ROOT / "models" / "scaler.pkl"
FEATURE_COLUMNS_PATH = PROJECT_ROOT / "models" / "feature_columns.pkl"
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "df_optimized.csv"  # CSV original

def load_artifacts():
    """Cargar modelo, scaler y columnas desde archivos .pkl"""
    with open(MODEL_PATH, 'rb') as f:
        model = pickle.load(f)
    with open(SCALER_PATH, 'rb') as f:
        scaler = pickle.load(f)
    with open(FEATURE_COLUMNS_PATH, 'rb') as f:
        feature_columns = pickle.load(f)
    return model, scaler, feature_columns

def generate_random_test_data(df_original, feature_columns, n_samples=5):
    """Genera datos aleatorios basados en estadísticas del dataset original"""
    random_data = {}
    for col in feature_columns:
        if df_original[col].dtype in ['int64', 'float64']:
            # Para numéricas: valor dentro del rango [Q1, Q3] (evita outliers)
            q1 = df_original[col].quantile(0.25)
            q3 = df_original[col].quantile(0.75)
            random_data[col] = [random.uniform(q1, q3) for _ in range(n_samples)]
        elif df_original[col].dtype == 'object':
            # Para categóricas: muestra aleatoria de valores únicos
            random_data[col] = random.choices(df_original[col].dropna().unique(), k=n_samples)
        else:
            # Para booleanas (ej: host_is_superhost): 0 o 1
            random_data[col] = [random.randint(0, 1) for _ in range(n_samples)]
    return pd.DataFrame(random_data)

def predict_price(model, scaler, X_test):
    """Preprocesa y predice precios"""
    # Escalar features numéricas (si el scaler existe)
    numeric_cols = X_test.select_dtypes(include=['int64', 'float64']).columns
    if scaler:
        X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])
    # Predecir y convertir a EUR
    return np.expm1(model.predict(X_test))

if __name__ == "__main__":
    # --- Cargar artefactos ---
    model, scaler, feature_columns = load_artifacts()
    print(f"🔄 Columnas esperadas: {feature_columns}")

    # --- Generar datos aleatorios consistentes ---
    df_original = pd.read_csv(DATA_PATH)
    X_test_random = generate_random_test_data(df_original, feature_columns, n_samples=3)
    
    print("\n🎲 Datos de prueba generados (aleatorios pero consistentes):")
    print(X_test_random)

    # --- Predecir ---
    prices = predict_price(model, scaler, X_test_random)
    for i, price in enumerate(prices):
        print(f"\n🏠 Predicción {i+1}: ${price:.2f} EUR")

    # --- Opcional: Exportar datos de prueba a CSV ---
    X_test_random.to_csv(PROJECT_ROOT / "data" / "test_samples.csv", index=False)
    print("\n💾 Datos guardados en 'data/test_samples.csv'")
```

---

### **¿Cómo funciona?**
1. **Generación de datos aleatorios**:
   - **Para features numéricas**: Usa el rango entre el primer y tercer cuartil (Q1-Q3) para evitar outliers.
   - **Para categóricas**: Muestra aleatoria de valores únicos del dataset original.
   - **Para booleanas**: Aleatoriamente 0 o 1.

2. **Validación implícita**:
   - Al basarse en `feature_columns.pkl`, garantiza que las columnas y orden sean idénticos a los de entrenamiento.

3. **Consistencia**:
   - Los datos generados respetan la distribución original (ej: `bedrooms` no será 100 si el máximo original era 5).

---

### **Ejemplo de salida**
```
🔄 Columnas esperadas: ['accommodates', 'bathrooms', 'bedrooms', 'beds', ...]

🎲 Datos de prueba generados:
   accommodates  bathrooms  bedrooms  beds  ...
0             2        1.0         1     1
1             4        2.0         2     3
2             3        1.5         1     2

🏠 Predicción 1: $85.50 EUR
🏠 Predicción 2: $120.30 EUR
🏠 Predicción 3: $92.10 EUR
```

---

### **Ventajas de este enfoque**
1. **Robustez**: Datos aleatorios pero realistas.
2. **Debugging fácil**: Puedes guardar los datos generados en CSV (`test_samples.csv`) para replicar pruebas.
3. **Preparación para producción**: Si luego quieres probar con datos reales, solo sustituye `X_test_random` por tu DataFrame.

---

### **¿Cómo personalizarlo?**
- **Para ajustar aleatoriedad**: Modifica `generate_random_test_data()`.
  - Ejemplo: Si `bathrooms` debe ser múltiplo de 0.5:
    ```python
    random_data[col] = [round(random.uniform(q1, q3) * 2) / 2 for _ in range(n_samples)]
    ```
- **Para excluir columnas**: Filtra `feature_columns` antes de usarlo.

¿Necesitas ajustar algo más específico para tu caso de uso?

## **2. Carga del Modelo.pkl con introducción de datos por usuario en terminal**

### **Script `test.py` para Ingresar Datos por Terminal (y Futura Integración con React)**

Aquí tienes un script que:
1. **Carga las columnas del modelo** desde `feature_columns.pkl`.
2. **Pide al usuario que ingrese valores para cada columna** (validando tipos de datos).
3. **Genera predicciones** y las devuelve en formato JSON (ideal para conectar con React).

---

#### **Archivo `test.py`**
```python
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import json

# --- Configuración de paths ---
PROJECT_ROOT = Path(__file__).parent.parent  # Ajusta según tu estructura
MODEL_PATH = PROJECT_ROOT / "models" / "random_forest_v5.pkl"
SCALER_PATH = PROJECT_ROOT / "models" / "scaler.pkl"
FEATURE_COLUMNS_PATH = PROJECT_ROOT / "models" / "feature_columns.pkl"

def load_artifacts():
    """Cargar modelo, scaler y columnas desde archivos .pkl"""
    with open(MODEL_PATH, 'rb') as f:
        model = pickle.load(f)
    with open(SCALER_PATH, 'rb') as f:
        scaler = pickle.load(f)
    with open(FEATURE_COLUMNS_PATH, 'rb') as f:
        feature_columns = pickle.load(f)
    return model, scaler, feature_columns

def get_user_input(feature_columns):
    """Recolecta datos de entrada del usuario vía terminal"""
    input_data = {}
    print("\n🛠️  Ingresa los valores para cada feature (presiona Enter para omitir y usar valor por defecto):")
    
    # Valores por defecto (basados en medianas o modas)
    default_values = {
        'accommodates': 2,
        'bathrooms': 1.0,
        'bedrooms': 1,
        'beds': 1,
        'minimum_nights': 2,
        'number_of_reviews': 10,
        'review_scores_rating': 90,
        'host_is_superhost': 0,
        'neighbourhood_density': 0.5,
        'has_wifi': 1,
        'has_air_conditioning': 1
    }
    
    for col in feature_columns:
        while True:
            try:
                user_input = input(f"{col} (default={default_values.get(col, 'N/A')}): ").strip()
                if not user_input:  # Si el usuario presiona Enter
                    input_data[col] = default_values[col]
                    break
                # Conversión de tipos
                if isinstance(default_values.get(col), float):
                    input_data[col] = float(user_input)
                elif isinstance(default_values.get(col), int):
                    input_data[col] = int(user_input)
                else:
                    input_data[col] = user_input
                break
            except ValueError:
                print(f"⚠️ Error: Ingresa un valor válido para {col} (ej: {default_values.get(col)})")
    
    return pd.DataFrame([input_data])

def predict_price(model, scaler, input_data):
    """Preprocesa y predice precios"""
    # Escalar features numéricas
    numeric_cols = input_data.select_dtypes(include=['int64', 'float64']).columns
    if scaler:
        input_data[numeric_cols] = scaler.transform(input_data[numeric_cols])
    # Predecir y convertir a EUR
    return np.expm1(model.predict(input_data))[0]

if __name__ == "__main__":
    # --- Cargar modelo y columnas ---
    model, scaler, feature_columns = load_artifacts()
    print(f"🔍 Columnas requeridas: {feature_columns}")
    
    # --- Obtener datos del usuario ---
    user_data_df = get_user_input(feature_columns)
    
    # --- Predecir y mostrar resultado ---
    predicted_price = predict_price(model, scaler, user_data_df)
    print(f"\n🎯 Precio predicho: ${predicted_price:.2f} EUR")
    
    # --- Salida en JSON (para integrar con React) ---
    output_json = {
        "input_data": user_data_df.iloc[0].to_dict(),
        "predicted_price": round(float(predicted_price), 2)
    }
    print("\n📤 JSON para React (copiar esto):")
    print(json.dumps(output_json, indent=2))
```

---

### **Cómo Usarlo**
1. **Ejecuta el script**:
   ```bash
   python test.py
   ```

2. **Ingresa valores** (o presiona Enter para usar valores por defecto):
   ```
   accommodates (default=2): 3
   bathrooms (default=1.0): 1.5
   bedrooms (default=1): 
   ...
   ```

3. **Resultado en terminal**:
   ```json
   {
     "input_data": {
       "accommodates": 3,
       "bathrooms": 1.5,
       "bedrooms": 1,
       ...
     },
     "predicted_price": 120.50
   }
   ```

---

### **Claves para la Integración con React**
1. **Formato JSON**:  
   El output está listo para ser consumido por tu frontend (ej: mediante `fetch` o Axios).

2. **Valores por defecto**:  
   Usa medianas/modes del dataset para evitar inputs vacíos.

3. **Validación de tipos**:  
   El script fuerza `int`/`float` según lo que el modelo espera.

4. **Escalabilidad**:  
   Si añades más features, solo actualiza `default_values`.

---

### **Adaptación para API (Opcional)**
Si prefieres llamar al script desde React como una API, usa `Flask`:

#### **Archivo `api.py`** (en la misma carpeta que `test.py`)
```python
from flask import Flask, request, jsonify
import subprocess

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    # Ejecuta test.py y captura el output
    result = subprocess.run(['python', 'test.py'], input=json.dumps(data), capture_output=True, text=True)
    return jsonify(json.loads(result.stdout))

if __name__ == "__main__":
    app.run(port=5000)
```

En React:
```javascript
fetch('http://localhost:5000/predict', {
  method: 'POST',
  headers: { 'Content-Type': 'application/json' },
  body: JSON.stringify({ accommodates: 2, bathrooms: 1.5, ... })
})
```

---

### **Notas Finales**
- **Seguridad**: Si el frontend y backend están separados, habilita CORS en Flask.
- **Performance**: Para producción, carga el modelo una vez al iniciar la API (no en cada request).
- **Testing**: Usa `pytest` para automatizar pruebas con diferentes inputs.

¿Necesitas ajustar algo más para tu implementación específica?

## **Si necesitas volver a entrenar el modelo**

Para que todo funcione correctamente, **sí necesitas volver a ejecutar el script de entrenamiento** (`train_model.py`) y asegurarte de que guarde el archivo `feature_columns.pkl` con las columnas correctas. Aquí te detallo los pasos:

---

### **Pasos a Seguir**

1. **Ejecuta el script de entrenamiento**:
   ```bash
   python train_model.py
   ```
   Esto generará (o actualizará) los archivos:
   - `models/random_forest_v5.pkl` (modelo).
   - `models/scaler.pkl` (scaler).
   - `models/feature_columns.pkl` (columnas usadas).

2. **Verifica que el archivo `feature_columns.pkl` existe**:
   ```bash
   ls models/feature_columns.pkl
   ```
   Si no aparece, revisa que tu script de entrenamiento incluya este bloque:
   ```python
   # En train_model.py (justo después de definir X_train)
   with open(model_dir / "feature_columns.pkl", 'wb') as f:
       pickle.dump(list(X_train.columns), f)
   ```

3. **Prueba los scripts de predicción**:
   - **Para datos aleatorios**:
     ```bash
     python test_model_with_random_data.py
     ```
   - **Para entrada manual por terminal**:
     ```bash
     python test.py
     ```

---

### **¿Por qué es necesario?**
- **Consistencia**: El archivo `feature_columns.pkl` asegura que los scripts de predicción usen **exactamente las mismas columnas** que el modelo espera (en el mismo orden).
- **Evita errores**: Si no lo regeneras, podrías tener discrepancias si:
  - Modificaste las features en el dataset original (`df_optimized.csv`).
  - Cambiaste el preprocesamiento en `train_model.py`.

---

### **Consejos Adicionales**
1. **Si ya entrenaste el modelo previamente**:
   - Borra los archivos antiguos en `models/` antes de reentrenar:
     ```bash
     rm models/*.pkl
     ```

2. **Para debuggear**:
   - Imprime las columnas justo antes de guardarlas:
     ```python
     print("Columnas guardadas:", list(X_train.columns))
     ```

3. **Si usas Git**:
   - Asegúrate de que `models/feature_columns.pkl` esté en tu `.gitignore` si el modelo pesa mucho (y documenta cómo generarlo).

---

### **Ejemplo de Salida Esperada**
Al ejecutar `train_model.py`, deberías ver algo como:
```
✅ Modelo, scaler y columnas guardados en: models/
Columnas guardadas: ['accommodates', 'bathrooms', ..., 'has_air_conditioning']
```

---

### **Integración con React (Recordatorio)**
Cuando conectes el script `test.py` con tu frontend:
1. **Desde React**, llama al script via API (ej: con Flask/Express) o subproceso.
2. **Pasa los inputs** como JSON (como ya hiciste en el script).
3. **Asegúrate** de que las columnas en React coincidan con las de `feature_columns.pkl`.

¿Necesitas ayuda para ajustar algo más en el script de entrenamiento o en la conexión con React?